In [8]:
pip install pycountry

     |████████████████████████████████| 10.1MB 5.5MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=a823492af784845a1f79ebfc58686dd412a93167ad086de89bb38f1ce1666d6e
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry


In [9]:
import pandas as pd
import plotly.express as px
from bs4 import BeautifulSoup as bs
import requests
import json
import pycountry 
import urllib
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from plotly.offline import  iplot

In [82]:
url = "https://github.com/ayedaemon/RuckSack-Python/blob/master/log_analysis/access_log"

In [54]:
def html_to_df(url, clean_empty = False , attrs = {}, helper = None):
    html_content = requests.get(url).text
    soup = bs(html_content, "lxml")
    table = str(soup.find("table", attrs=attrs))
    df = pd.read_html(str(table))[0]
    if clean_empty :
        df = df.loc[:, ~df.isnull().all(axis = 0)] 
    if helper:
        df = helper(df)   
    return df

In [55]:
df = html_to_df(url,clean_empty=True)

In [56]:
def log_parser(str_):
    find = {}
    find['ip'] = str_.split()[0]
    find['RFC931'] = str_.split()[1]
    find['user'] = str_.split()[2]
    find['date'] = str_.split('[')[1].split()[0]
    find['gmt'] = str_.split('[')[1].split()[1].strip(']')
    try:
        if 'HTTP' in str_.split('"')[1].split()[-1]:
            find['action'] =  str_.split('"')[1].replace(str_.split('"')[1].split()[2],'').strip()
        else:
            find['action'] =  str_.split('"')[1].strip()
    except:
        find['action'] = '-'
    try:
        find['status'] = str_.split('"')[2].strip().split()[0]
    except:
        find['status'] = '-'
    try:
        find['size'] = str_.split('"')[2].strip().split()[1]
    except:
        find['size'] = '-'
    try:
        find['referrer'] = str_.strip().split('"')[3]
    except:
        find['referrer'] ='-'
    try:
        find['browser'] = str_.strip().split('"')[5]
    except:
        find['browser'] = '-'
        
    return find

In [57]:
def log_df(df,col_name ,columns = [] ):
    df = pd.DataFrame(list(df[col_name].apply(log_parser).values) )
    if len(df.columns) == len(columns):
        df.columns = columns
    return df

In [58]:
columns = ["User Address","RFC931","User","date","gmt","action","status","size","referrer","browser"]
df = log_df(df,1,columns)

In [59]:
df.head()

,User Address,RFC931,User,date,gmt,action,status,size,referrer,browser
0,127.0.0.1,-,-,15/Jan/2021:08:54:07,+0000,GET /,403,4006,-,curl/7.61.1
1,127.0.0.1,-,-,15/Jan/2021:08:55:23,+0000,GET /cgi-bin/weakform.py,403,199,-,curl/7.61.1
2,127.0.0.1,-,-,15/Jan/2021:09:01:38,+0000,GET /cgi-bin/weak_form.py,200,19,-,curl/7.61.1
3,172.17.0.1,-,-,15/Jan/2021:09:02:14,+0000,GET /,403,4288,-,Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko...
4,172.17.0.1,-,-,15/Jan/2021:09:02:15,+0000,GET /noindex/common/css/bootstrap.min.css,200,99548,http://172.17.0.2/,Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko...


**Checking XSS Attack**

In [60]:
def XSS_finder(str_):
    flag = 0
    l1 = ['<','>','\\','`']
    l2 = ['/',')','(']
    for char in l1:
        aasci_encoding = '%'+hex(ord(char)).replace('0x','')
        if (char in str_) or (aasci_encoding in str_):
            flag = 1
            break
    if flag != 1:
        try:
            req_str = ''.join(str_.split('?')[1:])
            for char in l2:
                aasci_encoding = '%'+hex(ord(char)).replace('0x','')
                if (char in req_str) or (aasci_encoding in req_str):
                    flag = 1
                    break
        except:
            pass
    return bool(flag)

In [61]:
df['XSS_attack'] = df['action'].apply(XSS_finder)

In [62]:
df[df.XSS_attack == True].head()

,User Address,RFC931,User,date,gmt,action,status,size,referrer,browser,XSS_attack
754,172.17.0.1,-,-,15/Jan/2021:10:05:35,+0000,GET /cgi-bin/weak_form.py?fname=asdasd%3C%2Fp%...,200,355,http://172.17.0.2/cgi-bin/weak_form.py?fname=s...,Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko...,True
757,172.17.0.1,-,-,16/Jan/2021:05:29:02,+0000,GET /cgi-bin/weak_form.py?fname=abcd&lname=111...,200,328,-,curl/7.74.0,True
758,172.17.0.1,-,-,16/Jan/2021:05:30:04,+0000,GET /cgi-bin/weak_form.py?fname=abcd&lname=111...,200,340,-,curl/7.74.0,True
759,172.17.0.1,-,-,16/Jan/2021:05:30:42,+0000,GET /cgi-bin/weak_form.py?fname=abcd&lname=111...,200,340,-,curl/7.74.0,True
760,172.17.0.1,-,-,16/Jan/2021:05:30:57,+0000,GET /cgi-bin/weak_form.py?fname=abcd&lname=111...,200,341,-,curl/7.74.0,True


**Getting Location data from User Address**

In [63]:
def add_location_data(df,column,keys):
    def loc_dict_maker(unq_series):
        def ip_loc(x):
            res = {}
            url = "https://geolocation-db.com/jsonp/"+x
            with urllib.request.urlopen(url) as url:
                data = json.loads(url.read().decode().split("(")[1].strip(")"))
            res = {"country_code":data["country_code"], 'latitude':data['latitude'],'longitude':data['longitude']}
            try:
                res['alpha_3'] = pycountry.countries.get(alpha_2=res["country_code"]).alpha_3
            except:
                res['alpha_3'] = 'Not found'
            return res

        result = {}
        for ip in tqdm(unq_series):
            result[ip] = ip_loc(ip)
        return result
    
    loc_dict = loc_dict_maker(df[column].unique())
    for key in keys:
        df[key] = df[column].apply(lambda x: loc_dict[x][key])
    return df    

In [64]:
keys = ['country_code', 'latitude', 'longitude', 'alpha_3']
df = add_location_data(df,'User Address',keys)
df.head()

,User Address,RFC931,User,date,gmt,action,status,size,referrer,browser,XSS_attack,country_code,latitude,longitude,alpha_3
0,127.0.0.1,-,-,15/Jan/2021:08:54:07,+0000,GET /,403,4006,-,curl/7.61.1,False,Not found,Not found,Not found,Not found
1,127.0.0.1,-,-,15/Jan/2021:08:55:23,+0000,GET /cgi-bin/weakform.py,403,199,-,curl/7.61.1,False,Not found,Not found,Not found,Not found
2,127.0.0.1,-,-,15/Jan/2021:09:01:38,+0000,GET /cgi-bin/weak_form.py,200,19,-,curl/7.61.1,False,Not found,Not found,Not found,Not found
3,172.17.0.1,-,-,15/Jan/2021:09:02:14,+0000,GET /,403,4288,-,Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko...,False,Not found,Not found,Not found,Not found
4,172.17.0.1,-,-,15/Jan/2021:09:02:15,+0000,GET /noindex/common/css/bootstrap.min.css,200,99548,http://172.17.0.2/,Mozilla/5.0 (X11; Linux x86_64; rv:84.0) Gecko...,False,Not found,Not found,Not found,Not found


**Top visited User**

In [65]:
def top_and_least_visited_user(df,n=1,m=1):
    result = {}
    result['Top_visited_user'] = list(df['User Address'].value_counts().head(n).index)
    result['least_visited_user'] = list(df['User Address'].value_counts().tail(m).index)
    return result

In [66]:
top_and_least_visited_user(df)

{'Top_visited_user': ['172.17.0.1'], 'least_visited_user': ['127.0.0.1']}

**Top and Least visited Page**

In [67]:
def top_and_least_visited_page(df,n=1,m=1):
    result = {}
    pages = df.action.apply(lambda x: x.split('?')[0])
    result['Top_visited_page'] = list(pages.value_counts().head(n).index)
    result['least_visited_page'] = list(pages.value_counts().tail(m).index)
    return result

In [68]:
top_and_least_visited_page(df)

{'Top_visited_page': ['GET /cgi-bin/weak_form.py'],
 'least_visited_page': ['GET /noindex/common/css/styles.css']}

**Hits v/s Time graph**

In [69]:
import plotly.express as px
def plot_hit_vs_time(dt_series,title,xlabel,ylabel):
    fig = px.line( x=dt_series.unique(), y=dt_series.value_counts(sort = False))
    fig.update_layout(
        title=title,
         xaxis_title=xlabel,
        yaxis_title=ylabel)
    fig.show()

In [70]:
dt_series = pd.to_datetime(df['date'], format = '%d/%b/%Y:%H:%M:%S')
plot_hit_vs_time(dt_series,"Hits v/s Time: Complete",'Time',"Hits")

In [71]:
plot_hit_vs_time(dt_series[dt_series.apply(lambda x: x.year == 2004)], "Hit v/s time for year 2004", "Time", "Hits")

In [72]:
plot_hit_vs_time(dt_series[dt_series.apply(lambda x: x.year == 2021)], "Hit v/s time for year 2021", "Time", "Hits")

**Plotting Individual File Hits v/s Time**

In [73]:
df_1 = html_to_df(url,clean_empty=True)
columns = ["User Address","RFC931","User","date","gmt","action","status","size","referrer","browser"]
df_1 = log_df(df_1,1,columns)

In [74]:
plot_hit_vs_time(df_1['date'], 'Hit v/s time', 'Time', 'Hits')

**Plotting Users on Map**

In [75]:
def geo_plotting(country_series,colorbar_title,title):
    data = dict(
            type = 'choropleth',
            locations = country_series.value_counts().index,
            z = country_series.value_counts().values,
            colorbar = {'title' : colorbar_title},
          )
    layout = dict(
        title = title,
        geo = dict(
                showframe = False,
                projection = {'type':'natural earth'}
        )
    )
    choromap = go.Figure(data = [data],layout = layout)
    iplot(choromap)

In [76]:
geo_plotting(df[df['alpha_3']!= 'Not found']['alpha_3'],"No. of Users","User's Geolocation Data")

**Plotting OS/Devices/Browsers**

In [77]:
def OS_dict(browser_series):
    os = ['Windows','Machintosh','Linux','Other']
    res_dict = {o:0 for o in os}
    for browser in browser_series:
        flag = 0
        for i in os[:-1]:
            if i in browser:
                res_dict[i]+=1
                flag = 1
                break
        if flag==0:
            res_dict['Other']+=1
    return res_dict

In [78]:
OS = OS_dict(df.browser)

In [79]:
px.pie( values=list(OS.values()), names=list(OS.keys()), title='OS Used to open Webpage')

In [80]:
def brwsr_dict(browser_series):
    brwsr = ['Firefox','Chrome','Opera','Edg','Other']
    res_dict = {o:0 for o in brwsr}
    for browser in browser_series:
        flag = 0
        for i in brwsr[:-1]:
            if i in browser:
                res_dict[i]+=1
                flag = 1
                break
        if flag==0:
            res_dict['Other']+=1
    return res_dict

In [81]:
BRWSR = brwsr_dict(df.browser)
px.pie( values=list(BRWSR.values()), names=list(BRWSR.keys()), title='Browser Used to open Webpage')